<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/papers_code/torch_recurrent_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import modules

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 2. 데이터셋

dtype = torch.float
- 텐서의 데이터 타입 지정
- `torch.float` PyTorch에서 부동 소수점 수(32비트 부동 소수점, torch.float32)

In [3]:
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

# 3. 단어 처리

word_list
- sentence를 " " 두고 하나로 결합
- 공백 기준으로 split
- 중복제거한 집합생성 `Java의 HashSet`
- 다시 list로 변환

word_dict
- word_dict: {'dog': 0, 'i': 1, 'cat': 2}
- number_dict: {0: 'dog', 1: 'i', 2: 'cat'}

In [4]:
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

# 4. RNN모델 하이퍼파라미터

batch_size
- 한 번의 에포크에서 처리할 문장의 개수

n_step
- RNN이 한 번에 처리할 입력 시퀀스의 길이
- 각 문장에서 마지막 단어를 예측하기 위해 그 앞의 두 단어 사용
- 현재 입력되는 문장이 단어 3개로 구성되어 있으므로 3-1 = 2

n_hidden
- 은닉층의 뉴런 수

In [5]:
batch_size = len(sentences)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈

# 5. 배치데이터 생성

In [17]:
word_dict

{'cat': 0,
 'like': 1,
 'you': 2,
 'coffee': 3,
 'hate': 4,
 'dog': 5,
 'love': 6,
 'i': 7,
 'milk': 8}

In [30]:
curr = [word_dict[n] for n in sentences[1].split()[:-1]]

print(np.eye(n_class)[curr])
print(word_dict[sentences[1].split()[-1]])


[[0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]]
3


In [34]:
def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    # sen = 'i like dog'
    # word = ['i', 'like', 'dog']
    # word[:-1] = ['i', 'like']
    word = sen.split()

    # i 에 해당하는 index 7, like 에 해당하는 index 1
    # [7, 1]
    input = [word_dict[n] for n in word[:-1]]
    # dog 에 해당하는 index 5
    # [5]
    target = word_dict[word[-1]]

    # One-Hot Encoding
    input_batch.append(np.eye(n_class)[input])
    target_batch.append(target)

  return input_batch, target_batch

'''
input_batch[1]
[[0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 target_batch[1]
 3
'''
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32)
# input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

# 6. 모델링

In [43]:
class TextRNN(nn.Module):
  def __init__(self):
    # nn.Module의 초기화 메서드 호출
    # TextRNN 클래스가 nn.Module의 기능을 상속
    # custom NN구조 정의
    super(TextRNN, self).__init__()

    # input_size는 입력단어의 개수. 원핫인코딩 차원
    # 뉴런을 30% 무작위 비활성화
    # input -> hidden layer / hidden layer -> hidden layer 가중치 관리
    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)

    # hidden layer -> output layer 가중치 관리
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  # 순전파
  def forward(self, hidden, X):
    # 일반적으로 (시퀀스 길이, 배치 크기, 피쳐 수) 형태의 입력
    # 그러나 데이터 준비 과정에서는 입력이 (배치 크기, 시퀀스 길이, 피쳐 수) 형태로 제공
    # 따라서 RNN에 입력하기 전에 텐서의 차원을 교환
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    # torch.mm은 행렬 곱셈 수행
    # 최종 은닉 상태 outputs과 가중치 행렬 self.W를 곱하고, 편향 self.b를 더해 최종 출력값을 계산
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

# 7. 학습

In [46]:
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  # 가중치 초기화가 아니라 은닉상태를 초기화하는 것
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print(output)
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

tensor([[ 1.4540, -0.9499,  0.4646, -0.6645, -0.8106,  4.4832, -2.3059, -3.0883,
          0.6635],
        [-1.2431, -2.8590, -2.4866,  1.4203, -2.6205, -1.8600, -2.5311, -1.1958,
          1.1660],
        [-1.5217, -2.6091, -2.4897,  1.3455, -2.3233, -0.7837, -2.5932, -1.3597,
          1.2142],
        [ 2.9414, -2.4777,  0.0978, -0.1847, -2.8857, -0.7146, -2.4623, -3.1710,
          0.8587],
        [-0.2505, -3.6807, -2.3999,  1.5084, -3.6936, -4.5352, -2.5290, -1.3570,
          1.2925],
        [-0.1997, -3.6387, -2.4835,  1.5290, -3.7252, -4.3112, -2.6384, -1.4968,
          1.3141]], grad_fn=<AddBackward0>)
Epoch: 0100 cost = 0.608090
tensor([[ 1.6242, -1.1194, -0.1992, -1.2140,  0.1391,  6.0031, -2.5098, -1.6199,
          1.1518],
        [ 0.3779, -4.3031, -3.1280,  2.8941, -2.2058, -3.0328, -3.3579,  0.3576,
          1.3267],
        [-3.2712, -2.9141, -3.3150,  1.1769, -2.1700, -0.3117, -3.1829, -1.5669,
          2.7915],
        [ 4.1994, -2.5368,  0.0124, -0.2119, -3

# 8. 예측

In [48]:
# 예측을 위해 다시 은닉상태 초기화
hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)

# data는 텐서의 데이터
# max는 1행에서의 최댓값을 차원 유지하며 가져옴
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]

# sqeeuze 텐서에서 차원이 1인 부분을 제거
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']
